In [6]:
# 读取照片
import torch
from torchvision import transforms
from torch import nn
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
#d2l替代
class Accumulator:
    """For accumulating sums over `n` variables."""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

argmax = lambda x, *args, **kwargs: x.argmax(*args, **kwargs)
astype = lambda x, *args, **kwargs: x.type(*args, **kwargs)
reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)
def accuracy(y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = argmax(y_hat, axis=1)
    cmp = astype(y_hat, y.dtype) == y
    return float(reduce_sum(astype(cmp, y.dtype)))
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')
# 图片地址
ROOT_TRAIN = r'F:\train'
ROOT_TEST = r'F:\test'
# 灰度图
train_transform = transforms.Compose([transforms.ToTensor(),transforms.Grayscale(1),])
test_transform = transforms.Compose([transforms.ToTensor(),transforms.Grayscale(1)])
# 读入图片
train_dataset = ImageFolder(ROOT_TRAIN,transform=train_transform)
test_dataset = ImageFolder(ROOT_TEST,transform=train_transform)
print(len(train_dataset))
print(len(test_dataset))
# 模型
net = nn.Sequential(
    # 这里，我们使用一个11*11的更大窗口来捕捉对象。
    # 同时，步幅为4，以减少输出的高度和宽度。
    # 另外，输出通道的数目远大于LeNet
    nn.Conv2d(1, 96, kernel_size=11, stride=4, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
    nn.Conv2d(96, 256, kernel_size=5, padding=2), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    # 使用三个连续的卷积层和较小的卷积窗口。
    # 除了最后的卷积层，输出通道的数量进一步增加。
    # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度
    nn.Conv2d(256, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 384, kernel_size=3, padding=1), nn.ReLU(),
    nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
    nn.MaxPool2d(kernel_size=3, stride=2),
    nn.Flatten(),
    # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过度拟合
    nn.Linear(256, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(4096, 4096), nn.ReLU(),
    nn.Dropout(p=0.5),
    # 最后是输出层，输出为5类
    nn.Linear(4096, 5))

# 评估准确率
def evaluate_accuracy_gpu(net, data_iter, device=None): #@save
    """使用GPU计算模型在数据集上的精度。"""
    if isinstance(net, torch.nn.Module):
        net.eval()  # 设置为评估模式
        if not device:
            device = next(iter(net.parameters())).device
    # 正确预测的数量，总预测的数量
    metric = Accumulator(2)
    for X, y in data_iter:
        if isinstance(X, list):
            # BERT微调所需的（之后将介绍）
            X = [x.to(device) for x in X]
        else:
            X = X.to(device)
        y = y.to(device)
        metric.add(accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]
# 训练函数
def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
    """用GPU训练模型(在第六章定义)。"""
    # 初始化权重
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    # 判断是否在gpu上跑
    print('training on', device)
    net.to(device)
    # 优化函数
    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    # 损失函数
    loss = nn.CrossEntropyLoss()
    for epoch in range(num_epochs):
        # 训练损失之和，训练准确率之和，范例数
        # 累加器
        metric = Accumulator(3)
        net.train()
        print("开始训练")
        for  i,(X, y) in enumerate(train_iter):
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], accuracy(y_hat, y), X.shape[0])
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        print("第",epoch,"训练结果")
        print("损失")
        print(train_l)
        print("训练准确率")
        print(train_acc)
        print("测试准确率")
        print(test_acc)
# 超参数        
lr, num_epochs, batch_size = 0.001, 20, 512
# 加载训练数据
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)
train_ch6(net,train_dataloader,test_dataloader,num_epochs,lr,try_gpu())

14084
3751
training on cuda:0
开始训练
第 0 训练结果
损失
1.104824456190799
训练准确率
0.5937233740414655
测试准确率
0.7339376166355639
开始训练
第 1 训练结果
损失
0.715353360402249
训练准确率
0.7744958818517467
测试准确率
0.7797920554518795
开始训练
第 2 训练结果
损失
0.6049130674588075
训练准确率
0.7992047713717694
测试准确率
0.790455878432418
开始训练
第 3 训练结果
损失
0.5918082486699502
训练准确率
0.80168986083499
测试准确率
0.7880565182617969
开始训练
第 4 训练结果
损失
0.5157822709434312
训练准确率
0.8292388525986936
测试准确率
0.8101839509464143
开始训练
第 5 训练结果
损失
0.46073181837060784
训练准确率
0.8436523714853734
测试准确率
0.8285790455878432
开始训练
第 6 训练结果
损失
0.4324724760220762
训练准确率
0.8503976143141153
测试准确率
0.8349773393761664
开始训练
第 7 训练结果
损失
0.3910894835411707
训练准确率
0.8596279466060778
测试准确率
0.8333777659290855
开始训练
第 8 训练结果
损失
0.3945573380439669
训练准确率
0.8630360692984947
测试准确率
0.7797920554518795
开始训练
第 9 训练结果
损失
0.4453969477117485
训练准确率
0.8481965350752627
测试准确率
0.8400426552919221
开始训练
第 10 训练结果
损失
0.35401766505372756
训练准确率
0.8719823913660891
测试准确率
0.8525726472940549
开始训练
第 11 训练结果
损失
0.306408